In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler,SMOTE
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV  # For probability outputs
from sklearn.multiclass import OneVsRestClassifier

In [7]:
X_train = pd.read_csv('C:/Users/udin0/OneDrive/Documents/GitHub/Gazebo/Data/X_train.csv')
y_train = pd.read_csv('C:/Users/udin0/OneDrive/Documents/GitHub/Gazebo/Data/y_train.csv')
X_test_1 = pd.read_csv('C:/Users/udin0/OneDrive/Documents/GitHub/Gazebo/Data/X_test_1.csv')
X_test_2 = pd.read_csv('C:/Users/udin0/OneDrive/Documents/GitHub/Gazebo/Data/X_test_2.csv')
y_test_2_reduced = pd.read_csv('C:/Users/udin0/OneDrive/Documents/GitHub/Gazebo/Data/y_test_2_reduced.csv')

In [8]:
# Split original training data into train/validation sets
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

# 1. Check class distribution
class_counts = Counter(y_train_split)
print("Class counts:", class_counts)

# 2. Safe SMOTE setup
min_class_count = min(class_counts.values())
safe_k_neighbors = max(1, min(5, min_class_count - 1))

if min_class_count <= 5:
    print("Using RandomOverSampler for tiny classes.")
    sampler = RandomOverSampler(random_state=42)
else:
    print("Using SMOTE with k_neighbors =", safe_k_neighbors)
    sampler = SMOTE(random_state=42, k_neighbors=safe_k_neighbors)

# 3. Resample
X_train_res, y_train_res = sampler.fit_resample(X_train_split, y_train_split)
print("Resampled class counts:", Counter(y_train_res))

# Scale features (critical for SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_res)
X_val_scaled = scaler.transform(X_val)

Class counts: Counter({'label': 1})
Using RandomOverSampler for tiny classes.
Resampled class counts: Counter({'label': 1})


In [9]:
svm_model = OneVsRestClassifier(
    CalibratedClassifierCV(
        LinearSVC(
            C=1.0,  # Default regularization
            class_weight='balanced',
            dual=False,
            max_iter=10000,
            random_state=42
        ),
        cv=3,
        method='sigmoid'
    ),
    n_jobs=-1
)

In [10]:
# Train the model
svm_model.fit(X_train_scaled, y_train_res)

OSError: [WinError 1450] Insufficient system resources exist to complete the requested service

In [ ]:
y_pred = svm_model.predict(X_val_scaled)
y_proba = svm_model.predict_proba(X_val_scaled)

print("\n=== Model Performance ===")
print(classification_report(y_val, y_pred))
print(f"Macro F1: {f1_score(y_val, y_pred, average='macro'):.4f}")
print(f"Weighted F1: {f1_score(y_val, y_pred, average='weighted'):.4f}")